## Introduction to Transformers

In this notebook, we"re going to install a transformer model, analyze the embedding output, and compare some vectors

In [ ]:
from aips import *

import numpy
import sentence_transformers
from IPython.display import display, HTML

## Listing 13.5


In [2]:
from sentence_transformers import SentenceTransformer
transformer = SentenceTransformer("roberta-base-nli-stsb-mean-tokens")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Listing 13.6

In [3]:
phrases = ["it's raining hard", "it is wet outside",
           "cars drive fast", "motorcycles are loud"]
embeddings = transformer.encode(phrases, convert_to_tensor=True)
print("Number of embeddings:", len(embeddings))
print("Dimensions per embedding:", len(embeddings[0]))
print("The embedding feature values of \"it's raining hard\":")
print(embeddings[0])

Number of embeddings: 4
Dimensions per embedding: 768
The embedding feature values of "it's raining hard":
tensor([ 1.1609e-01, -1.8422e-01,  4.1023e-01,  2.8474e-01,  5.8746e-01,
         7.4418e-02, -5.6910e-01, -1.5300e+00, -1.4629e-01,  7.9517e-01,
         5.0953e-01,  3.5076e-01, -6.7288e-01, -2.9603e-01, -2.3220e-01,
         4.8475e-01,  9.9531e-01,  6.1437e-01,  7.8995e-02, -7.7781e-01,
         1.0021e+00,  3.5468e-01, -1.7308e-01,  3.9410e-01,  1.6540e-01,
        -1.2335e-01,  6.1811e-01,  3.6482e-01,  3.2900e-01,  1.0812e+00,
        -5.4269e-01, -2.2409e-01, -1.4409e+00,  8.2625e-01, -1.1814e+00,
        -4.4101e-01,  5.8892e-01, -1.5328e+00, -2.9688e-01, -6.7421e-03,
         4.8688e-01,  1.0616e-01, -2.7084e-01,  2.4105e-02,  2.8154e-01,
        -2.0851e-01, -2.3183e-01, -1.0750e+00, -6.3038e-01, -7.4111e-02,
         3.0137e-01, -1.2426e+00, -2.5925e-01, -3.2736e-02,  9.0476e-01,
         1.0643e-02,  1.2886e-01,  8.2835e-01, -1.5106e+00, -4.7442e-01,
         1.5537e+

## Listing 13.7


In [4]:
def normalize_embedding(embedding):
    normalized = numpy.divide(embedding, numpy.linalg.norm(embedding))
    return list(map(float, normalized))

normalized_embeddings = list(map(normalize_embedding, embeddings))
similarities = sentence_transformers.util.dot_score(normalized_embeddings,
                                                    normalized_embeddings)
print("The shape of the resulting similarities:", similarities.shape)

The shape of the resulting similarities: torch.Size([4, 4])


## Listing 13.8

In [5]:
def rank_similarities(phrases, similarities):
    a_phrases = []
    b_phrases = []
    scores = []
    for a in range(len(similarities) - 1):
        for b in range(a + 1, len(similarities)):
            a_phrases.append(phrases[a])
            b_phrases.append(phrases[b])
            scores.append(float(similarities[a][b]))
    dataframe = pandas.DataFrame({"score": scores,
                                  "phrase a": a_phrases, "phrase b": b_phrases})
    dataframe["idx"] = dataframe.index
    dataframe = dataframe.reindex(columns=["idx", "score", "phrase a", "phrase b"])
    return dataframe.sort_values(by=["score"], ascending=False,
                                    ignore_index=True)

dataframe = rank_similarities(phrases, similarities)
display(HTML(dataframe.to_html(index=False)))

idx,score,phrase a,phrase b
0,0.669060,it's raining hard,it is wet outside
5,0.590783,cars drive fast,motorcycles are loud
1,0.281166,it's raining hard,cars drive fast
2,0.280800,it's raining hard,motorcycles are loud
4,0.204867,it is wet outside,motorcycles are loud
3,0.138172,it is wet outside,cars drive fast


Up next: [Natural Language Autocomplete](3.natural-language-autocomplete.ipynb)